In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
import sys, os
sys.path.append('..')

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from config.config import cfg
from utils.load_data import load_data
from utils.plot import sample_plot

# DATA PIPELINE

In [ ]:
# VARIABLE
IMG_SIZE = cfg['image']['image_size']
BATCH_SIZE = cfg['image']['batch_size']
SEED =  cfg['image']['seed']
REGIONAL_DATA = os.path.join(os.path.dirname(os.getcwd()), 'region_proposal')

#### Data loading

In [ ]:
# Loading the processed data
train_ds, val_ds, test_ds = load_data(REGIONAL_DATA, BATCH_SIZE, IMG_SIZE, SEED)
sample_plot(train_ds, 10)

##### Preprocessing (Normalisation, Augmentation)

In [ ]:
data_preprocessing = keras.Sequential([
    # Rescaling
    keras.layers.Rescaling(1./255),

    # Augmentation
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomRotation(.1), # ~ +- 10% rotation
    keras.layers.RandomZoom(.1),     # zoom in/out
    keras.layers.RndomContrast(.1),  # adjusting contrast
    keras.layers.RandomBrightness(factor=.1), # adjusting brightness
    keras.layers.GaussianNoise(.01)  # adding light noise
])

#### Model building

In [ ]:
model = keras.Sequential(
    data_preprocessing,
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Flatten(),

    layers.Dense(128, activation='relu'),
    layers.Dense(8, activation='softmax')
)